In [16]:
import os
import requests
from bardapi import BardCookies
from sydney import SydneyClient
from dotenv import load_dotenv
import markdown
import re
import random
import string

load_dotenv()

True

In [3]:
models = [
    ('summarize', 'facebook/bart-large-cnn'),
    ('summarize', 'google/pegasus-large'),
    ('summarize', 'marianna13/flan-t5-base-summarization'),
    ('toxic', 's-nlp/roberta_toxicity_classifier'),
    ('toxic', 'citizenlab/distilbert-base-multilingual-cased-toxicity', 'inputs'),
    ('toxic', 'martin-ha/toxic-comment-model'),
    ('spam', 'rafacost/bert_base_pt_en_cased_email_spam'),
    ('spam', 'h-e-l-l-o/email-spam-classification-merged'),
    ('spam', 'dima806/email-spam-detection-roberta'),
    ('translate', 't5-base'),
    ('translate', 'allenai/wmt16-en-de-12-1'),
    ('translate', 'facebook/wmt19-en-de'),
    ('fillmask', 'bert-base-uncased', '[MASK]'),
    ('fillmask', 'vinai/bertweet-base', '<mask>'),
    ('fillmask', 'roberta-base', '<mask>'),
]

In [4]:
def request_to_model(model, input):

    BASE_URL = "https://api-inference.huggingface.co/models/"
    headers = {"Authorization": f"Bearer {os.getenv('HUGGING_FACE_API_KEY')}"}
    new_input = input
    parameters = {}

    if model[0] == 'fillmask':
        new_input = input.replace('<mask>', model[2])
    elif model[0] == 'translate':
        parameters = {"src_lang": "en_XX", "tgt_lang": "tgt_XX"}

    query = {"inputs": new_input, "wait_for_model": True, "parameters":parameters}

    try:
        response = requests.post(BASE_URL + model[1], json=query, headers=headers)
    except Exception as exception:
        print(exception)
    
    return response.json()

In [9]:
res = request_to_model(models[10], "The tower is 324 metres (1,063 ft) tall") # "The goal of life is <mask>."
print(res)

[{'translation_text': 'Das Handtuch ist 324 Meter groß.'}]


In [10]:
def convert_to_plain_text(input_text):
    input_text = re.sub(r'\[\^\d+\^\]', '', input_text)
    plain_text = markdown.markdown(input_text, output_format='html')
    plain_text = re.sub(r'<[^>]*>\s*', '', plain_text)
    return plain_text

In [121]:
def get_only_answer_bing(input):
    indice_inicio = input.find(":") 
    
    if indice_inicio != -1:
        indice_inicio += 1 

        return input[indice_inicio:].strip()
    else:
        return input

In [141]:
async def request_to_bing(question, type="q&a"):

    prompt = ""

    if (type == "q&a"):
        prompt = "Answer me the following question in plain text without using quotes: "
    elif (type == "change_order"):
        prompt = "Change the order of the next sentence: "
    elif (type == "add_random_words"):
        prompt = "Add random words, do not replace, to the next sentence but don't let the phrase lose its meaning: "
    elif (type == "remplace_named_entities"):
        prompt = "Replace all named entities to alter the demographic context of the following sentence, give me only the new sentence: "
    elif (type == "use_negation"):
        prompt = "Modify the following sentence using negation, give me only the new sentence: "

    async with SydneyClient(style="precise") as sydney:
        response = await sydney.ask(prompt + question, citations=False)
        response = convert_to_plain_text(response)
        return response

In [10]:
def request_to_bard(question):

    cookie_dict = {
        "__Secure-1PSID": os.getenv('SECURE_1PSID'),
        "__Secure-1PSIDTS": os.getenv('SECURE_1PSIDTS'),
        "__Secure-1PSIDCC": os.getenv('SECURE_1PSIDCC'),
    }

    bard = BardCookies(cookie_dict=cookie_dict)
    response = bard.get_answer(question)['content']
    return response

In [13]:
print(request_to_bard("When was Bard released?"))

Bard was initially announced on February 17, 2023, and released in a limited capacity on March 21, 2023. It was upgraded to PaLM and then to Gemini in 2023.


PERTURBACIONES

In [49]:
def replace_characters(input, nivel):
    if nivel < 1 or nivel > 10:
       return "Level must be between 1 and 10."
    
    caracteres = list(input)
    indices_a_sustituir = [i for i in range(len(caracteres)) if caracteres[i] in string.ascii_letters]
    num_caracteres_a_sustituir = int(len(indices_a_sustituir) * nivel / 20)
    indices_a_sustituir = random.sample(indices_a_sustituir, num_caracteres_a_sustituir)
    
    for indice in indices_a_sustituir:
        caracteres[indice] = random.choice(string.ascii_letters)
    
    return ''.join(caracteres)


In [52]:
print(replace_characters("Hello how are you? It's been a while since you and I met.", 5))

Hehlo ylw are doF? It'i Sqen a while Oince you anf I met.


In [53]:
def add_characters(input, level):
    if level < 1 or level > 10:
        return "Level must be between 1 and 10."
    
    characters = list(input)
    num_characters_to_add = int(len(characters) * level / 20)
    
    for _ in range(num_characters_to_add):
        index = random.randint(0, len(characters))
        characters.insert(index, random.choice(string.ascii_letters))
    
    return ''.join(characters)


In [54]:
print(add_characters("Hello how are you? It's been a while since you and I met.", 4))

HEello how are PyouX? Iwt's beEen a while wsincbeYZ ynou and oI met.


In [64]:
async def add_random_words(input):
    response = await request_to_bing(input, "add_random_words")
    return response

In [160]:
text_random_words = await add_random_words("Hello how are you? It's been a while since you and I met.")
print(text_random_words)

Hello, how are you doing today? It's been quite a while since the last time you and I had the chance to meet.


In [94]:
async def remplace_named_entities(input):
    response = await request_to_bing(input, "remplace_named_entities")
    return get_only_answer_bing(response)

In [159]:
text_remplace_named_entities = await remplace_named_entities("Tomas and Maria are from Spain. They are 20 years old. They are married. They have two children.")
print(text_remplace_named_entities)

Akio and Yumi are from Japan. They are 30 years old. They are married. They have three children.


In [155]:
async def change_order(input):
    response = await request_to_bing(input, "change_order")
    return get_only_answer_bing(response)

In [158]:
text_change_order = await change_order("Hello how are you? It's been a while since you and I met.")
print(text_change_order)

It's been a while since you and I met. Hello, how are you?


In [151]:
async def use_negation(input):
    response = await request_to_bing(input, "use_negation")
    return get_only_answer_bing(response)

In [153]:
text_use_negation = await use_negation("Hello how are you? It's been a while since you and I met. I like to eat pizza.")
print(text_use_negation)

Hello, how are you? It hasn't been long since you and I met. I don't like to eat pizza.
